In [1]:
import cv2
import numpy as np
import os


def image_stats(image):
    # Compute the mean and standard deviation of each channel
    (l, a, b) = cv2.split(image)
    (lMean, lStd) = (l.mean(), l.std())
    (aMean, aStd) = (a.mean(), a.std())
    (bMean, bStd) = (b.mean(), b.std())
 
    return (lMean, lStd, aMean, aStd, bMean, bStd)

def color_transfer(source, target):
    source = cv2.cvtColor(source, cv2.COLOR_BGR2LAB).astype("float32")
    target = cv2.cvtColor(target, cv2.COLOR_BGR2LAB).astype("float32")
    
    height, width, _ = source.shape
    
#[(299, 717), (700, 1152)]
    (lMeanSrc, lStdSrc, aMeanSrc, aStdSrc, bMeanSrc, bStdSrc) = image_stats(source[717:1152, 299:700])
    (lMeanTar, lStdTar, aMeanTar, aStdTar, bMeanTar, bStdTar) = image_stats(target[717:1152, 299:700])
    
    (l, a, b) = cv2.split(source)
    l -= lMeanSrc
    a -= aMeanSrc
    b -= bMeanSrc
 
    l = (lStdTar / lStdSrc) * l
    a = (aStdTar / aStdSrc) * a
    b = (bStdTar / bStdSrc) * b
 
    l += lMeanTar
    a += aMeanTar
    b += bMeanTar
 
    l = np.clip(l, 0, 255)
    a = np.clip(a, 0, 255)
    b = np.clip(b, 0, 255)
 
    transfer = cv2.merge([l, a, b])
    transfer = cv2.cvtColor(transfer.astype("uint8"), cv2.COLOR_LAB2BGR)
    
    return transfer

# Get a list of all image file names in the folder
mainDir = r"C:\Users\ahj28\Desktop\Python\Controls\1\Happy"
inDir= mainDir + "\EyeBlacked"
#outDir = r"C:\Users\ahj28\Desktop\Python\SampleOutput430sec\ColorOutput"
outDir = mainDir+"\ColorOutput"

if not os.path.exists(outDir):
    os.makedirs(outDir)


image_files = [f for f in os.listdir(inDir) if f.endswith('.png')]

# Sort the image files and get the first image as the reference
image_files.sort()
imageA = cv2.imread(os.path.join(inDir, image_files[0]))

#copy imageA(refence) to outDir
cv2.imwrite(os.path.join(outDir, image_files[0]),imageA)


# Perform color transfer for all other images
for image_file in image_files[1:]:
    imageB = cv2.imread(os.path.join(inDir, image_file))
    transfer = color_transfer(imageB, imageA)
    
    # Save the adjusted image using cv2.imwrite
    cv2.imwrite(os.path.join(outDir, image_file), transfer)
